## multiple linear regression 
1. USE Student_performance dataset 
2. preprocess data (train test split)
3. train MLR model 
4. find r2 score of model

In [68]:
from warnings import filterwarnings
filterwarnings('ignore')

In [69]:
import pandas as pd
df = pd.read_csv('Datasets\Student_Performance.csv')
df

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


### Performance Index ~ X
target is performance index

In [70]:
df.isna().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


In [72]:
X = df.drop(columns=['Performance Index'])
Y = df[['Performance Index']]
X.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced
0,7,99,Yes,9,1
1,4,82,No,4,2
2,8,51,Yes,7,2
3,5,52,Yes,5,2
4,7,75,No,8,5


In [73]:
Y.head()

,Performance Index
0,91.0
1,65.0
2,45.0
3,36.0
4,66.0


In [74]:
from PM6func import catconsep
cat, con = catconsep(X)
cat

['Extracurricular Activities']

In [75]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X_cat_pre = ohe.fit_transform(X[cat]).toarray()
X_cat_pre

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], shape=(10000, 2))

In [76]:
cols = ohe.get_feature_names_out()
cols

array(['Extracurricular Activities_No', 'Extracurricular Activities_Yes'],
      dtype=object)

In [77]:
X_cat_pre = pd.DataFrame(X_cat_pre, columns=[cols])
X_cat_pre.head()

,Extracurricular Activities_No,Extracurricular Activities_Yes
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0


In [78]:
X_con = X[con]

In [79]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_con_pre = scaler.fit_transform(X_con)
X_con_pre

array([[ 0.77518771,  1.70417565,  1.45620461, -1.24975394],
       [-0.38348058,  0.72391268, -1.49229423, -0.90098215],
       [ 1.16141048, -1.06362569,  0.27680507, -0.90098215],
       ...,
       [ 0.38896495,  0.7815752 ,  0.86650484,  0.1453332 ],
       [ 1.54763324,  1.5888506 ,  0.27680507, -1.59852572],
       [ 0.77518771,  0.26261245,  0.86650484, -1.24975394]],
      shape=(10000, 4))

In [80]:
con_cols = scaler.get_feature_names_out()
con_cols

array(['Hours Studied', 'Previous Scores', 'Sleep Hours',
       'Sample Question Papers Practiced'], dtype=object)

In [81]:
X_con_pre = pd.DataFrame(X_con_pre, columns=[con_cols])
X_con_pre.head(
)

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced
0,0.775188,1.704176,1.456205,-1.249754
1,-0.383481,0.723913,-1.492294,-0.900982
2,1.161410,-1.063626,0.276805,-0.900982
3,0.002742,-1.005963,-0.902594,-0.900982
4,0.775188,0.320275,0.866505,0.145333


In [82]:
X_pre = X_con_pre.join(X_cat_pre)
X_pre.head()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Extracurricular Activities_No,Extracurricular Activities_Yes
0,0.775188,1.704176,1.456205,-1.249754,0.0,1.0
1,-0.383481,0.723913,-1.492294,-0.900982,1.0,0.0
2,1.161410,-1.063626,0.276805,-0.900982,0.0,1.0
3,0.002742,-1.005963,-0.902594,-0.900982,0.0,1.0
4,0.775188,0.320275,0.866505,0.145333,1.0,0.0


In [83]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_pre,Y, test_size=0.2, random_state=21)
xtrain.shape

(8000, 6)

In [84]:
xtest.shape

(2000, 6)

In [85]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xtrain,ytrain)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


### R2 Score of train and test model

In [86]:
model.score(xtrain,ytrain)

0.988716967469377

In [87]:
model.score(xtest,ytest)

0.9888763071047979

In [88]:
y_pred_tr = model.predict(xtrain)
y_pred_tr[0:5]

array([[58.85858723],
       [46.24601559],
       [46.71989163],
       [74.7832757 ],
       [91.83819082]])

In [89]:
ytrain.head()

,Performance Index
5504,59.0
5307,49.0
1684,47.0
2441,74.0
0,91.0


In [90]:
y_pred_ts = model.predict(xtest)
y_pred_ts[0:5]

array([[73.91020091],
       [42.59597836],
       [34.37456296],
       [69.13540457],
       [36.73862157]])

In [91]:
ytest.head()

,Performance Index
3644,78.0
1828,46.0
6661,35.0
9216,71.0
5911,37.0


1. mean squared error
2. mean absolute error
3. root mean squared error

In [92]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mse = mean_squared_error(ytest, y_pred_ts)
mse

4.187913060997729

In [93]:
mae = mean_absolute_error(ytest,y_pred_ts)
mae

1.6294216366159109

In [94]:
rmse = mse**(1/2)
rmse


2.046439117344498